In [ ]:
#Generating Neccessary Functions 
import numpy as np
from PIL import Image
import cv2
import glob
from scipy import interpolate
from scipy.interpolate import CubicSpline
from matplotlib import pyplot as plt
import os
from pathlib import Path

from ripser import ripser 
from persim import plot_diagrams
from sklearn.decomposition import PCA

#topological data analysis
from persim import plot_diagrams
from dreimac import CircularCoords 
from dreimac import CircleMapUtils as CMU

#plotting and visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline  

In [ ]:
def introduction(): 
    intro = '''Welcome to our Topological Decoupling Simulation!  This simulation aims to apply a chain of algorithms 
    that, when given a quasi-periodic video with N independent oscillations, rearranges the frames in N different 
    ways such that for the nth video created, the nth oscillation will be in motion while the others remain stationary.
    We achieve this by applying the a sliding-window algorithm to provided video data to extract quasi-periodocity.
    We also make use of PCA for visualization purposes, ripser to detect persistent cohomology, and the DREiMac
    algorithm to apply circular coordinates.  
    For our simulation, we ask that you provide the following:
    '''
    N=int(input("Number of Oscillations present in video data: "))
    videofp=input("Please provide the file path to your video (include quotation marks): ")
    WindowSize=int(input("Window Size (a size of 15-18 is recommended): "))
    Dimension=int(input("Sliding window dimension (a dimension of 3-5 is recommended): "))
    SampleSize=int(input("Sample Size (size of Sliding Window sample, 300-500 is recommended): "))
    PCA_Components=int(input("Number of components for PCA: "))
    
    

In [ ]:
 #Generating Video Functions 
    def getSlidingWindowVideo(I, dim, Tau, dT):
        N = I.shape[0] #Number of frames
        P = I.shape[1] #Number of pixels (possibly after PCA)
        pix = np.arange(P)
        NWindows = int(np.floor((N-dim*Tau)/dT))
        X = np.zeros((NWindows, dim*P))
        idx = np.arange(N)
        for i in range(NWindows):
            idxx = dT*i + Tau*np.arange(dim)
            start = int(np.floor(idxx[0]))
            end = int(np.ceil(idxx[-1]))
            f = interpolate.interp2d(pix, idx[start:end+1], I[idx[start:end+1], :], kind='linear')
            X[i, :] = f(pix, idxx).flatten()
        return X

    def writeVideo(filename, frame_data, fps, resol ):
        n_row, n_col = resol
        out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (n_col, n_row))
        if frame_data.dtype !='uint8':
            frame_data -= np.amin(frame_data)
            frame_data *= 255/np.amax(frame_data)
            frame_data = np.uint8(frame_data)
        for frame in frame_data:
            out.write(frame.reshape((n_row, n_col,-1)))
        out.release()
    def playVideo(filepath):
        cap = cv2.VideoCapture(filepath)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('frame', frame)
            if cv2.waitKey(24) == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()  

    def loadVideo(filepath):
        vid = cv2.VideoCapture(filepath)
        _, image = vid.read()
        count = 0
        success = True
        n_row, n_col, _ = image.shape
        data = []
        while success:
            data.append(image.flatten()) 
            success,image = vid.read()
            count += 1
        vid.release() 
        data = np.array(data)
        print(count, " frames extracted")
        print("frame size = ", (n_row , n_col))
        print("data shape =", data.shape)
        return data , (n_row, n_col)

In [ ]:
#Storing Video Data into Data Variable
data, shape = loadVideo(videofp)

In [ ]:
#Performing Sliding Window Algorithm 
wSize = WindowSize
dim = Dimension
Tau = wSize/float(dim)
desiredSamples=SampleSize
M = data.shape[0] - wSize + 1
dT = M/float(desiredSamples)
X = getSlidingWindowVideo(data, dim, Tau, dT)
frame_data = X[:, 0:data.shape[1]]

In [ ]:
# Get the circular coordinates at birth, and for the N/2 or N/3 cocycles with highest persistence
coho_classes=[]
if N % 2==0: 
    for i in range(N/2): 
        coho_classes.append(i)
elif N=1: 
    for i in range (N): 
        coho_classes.append(i)
elif N % 3==0:
    for i in range(N/3): 
        coho_classes.append(i)
else: 
    for i in range(5): 
        coho_classes.append(i)

circular_coords = []

n_lands = 0.75*desiredSamples

cc = CircularCoords(XS, n_landmarks=n_lands)

for i in coho_classes:
    #perc: specifies the time in filtration to construct the coordinates 
    #cocyle_index: Picks a persistent homology class when the classes are ordered with respect to persistence
    theta = cc.get_coordinates(perc= 0, cocycle_idx=i)
    theta = CMU.to_sinebow(np.pi + CMU.center(theta))
    circular_coords.append(theta)



In [ ]:
# PCA of the Sliding window point cloud
pca = PCA(n_components=PCA_Components)
XS_pca = pca.fit(XS).transform(XS)
plt.figure(figsize = (3.8,1.5))
plt.plot(pca.explained_variance_ratio_, '-*') ;
plt.title('Explained PCA variance -Sliding Window');
    

In [ ]:
# Plot data with N circular coordinates

fig = make_subplots(rows=1, cols=len(coho_classes), 
                    subplot_titles = tuple('Circ Coord '+str(i+1) for i in coho_classes),
                    specs = [[{'type': 'scatter3d'} for i in coho_classes]])

for i, coord in enumerate(circular_coords):
    fig.add_trace(
        go.Scatter3d(x=XS_pca[:,0], y=XS_pca[:,1], z=XS_pca[:,2],   mode ='markers', 
        marker=dict(size = 2 , color =  circular_coords[i])),  
        row=1, col=i+1)
    fig.update_scenes(xaxis=dict( ticks='', showticklabels=False), 
                      yaxis=dict( ticks='', showticklabels=False), 
                      zaxis=dict( ticks='', showticklabels=False), 
                      aspectmode='data', row=1, col=i+1)

fig.update_layout( showlegend=False )
fig.show()


In [ ]:
#Gathering Angles from Circular Coordinates For the N-Oscillations variables = []

for i in range(N):
    variable_name = "theta" + str(i+1) + "vals"
    value = []  # Initialize an empty list
    variables.append((variable_name, value))
    # Appending unique elements to each variable's list
for i, variable in enumerate(variables):
    name, value = variable
    num_elements = len(circular_coords[0])
    elements = elements_list[i]

    for element in elements:
        if element not in value:
            value.append(element)

# Accessing the variables with the appended elements
for variable in variables:
    name, value = variable
    print(f"{name} = {value}")